In [1]:
import open3d as o3d
import os
import numpy as np
from sklearn.neighbors import kneighbors_graph
import maxflow
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
datasetPath = "/PublicData/CadDataset/a1.0.0_10"
part = "140294_f659b875"
name = "assembly.obj"
file_path = os.path.join(datasetPath,part,name)

In [3]:
# 读取网格
mesh = o3d.io.read_triangle_mesh(file_path)
if not mesh.has_vertices():
    raise RuntimeError("网格无顶点")
points = np.asarray(mesh.vertices)
print(f"读取网格顶点数: {len(points)}")

读取网格顶点数: 111342


In [4]:
def graphcut_pointcloud_segmentation(points, k=8):
    N = points.shape[0]
    print(f"点数: {N}")

    # 1. 构建k近邻图
    connectivity = kneighbors_graph(points, n_neighbors=k, mode='connectivity', include_self=False)
    connectivity = connectivity.tocoo()
    print(f"邻接边数: {connectivity.nnz}")

    # 2. 创建最大流图
    g = maxflow.Graph[float](N, connectivity.nnz)
    nodes = g.add_nodes(N)

    # 3. 添加边，权重基于距离（距离越小权重越大）
    row, col = connectivity.row, connectivity.col
    for i, j in zip(row, col):
        dist = np.linalg.norm(points[i] - points[j])
        weight = np.exp(-dist * 10)
        g.add_edge(i, j, weight, weight)

    # 4. 添加终端边，基于Z坐标简易Data term
    z = points[:, 2]
    median_z = np.median(z)
    for i in range(N):
        source_cap = max(0, median_z - z[i])  # 越低越靠近source（标签0）
        sink_cap = max(0, z[i] - median_z)    # 越高越靠近sink（标签1）
        g.add_tedge(i, source_cap, sink_cap)

    # 5. 计算最大流/最小割
    flow = g.maxflow()
    print(f"最大流: {flow}")

    # 6. 获取标签
    labels = np.array([g.get_segment(i) for i in range(N)])
    return labels

In [5]:
# 图割分割
labels = graphcut_pointcloud_segmentation(points, k=8)
print(f"分割出 {labels.max() + 1} 个类别")

点数: 111342
邻接边数: 890736
最大流: 0.0
分割出 2 个类别


In [6]:
# 顶点染色
num_clusters = labels.max() + 1
colors = np.zeros((len(labels), 3))
cmap = plt.get_cmap("tab10")
for i in range(num_clusters):
    colors[labels == i] = cmap(i / max(1, num_clusters - 1))[:3]
mesh.vertex_colors = o3d.utility.Vector3dVector(colors)

In [7]:
o3d.io.write_triangle_mesh("graphcut_segment.ply",mesh)

True